In [34]:
!sudo pip3 install facebook-sdk

In [35]:
import os
import re
import time
import facebook
import requests
import configparser
import urllib.request
import pandas as pd
from PIL import Image
from bs4 import BeautifulSoup
from datetime import datetime
from google.colab import drive

In [36]:
c=drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
IMG_URI = 'drive/My Drive/development/test/'
CSV_URI = 'drive/My Drive/development/test/'
CNF_INI = 'drive/My Drive/development/config/config.ini'

In [38]:
config = configparser.ConfigParser()
config.read(CNF_INI)
tips = config['facebook']['tips']
news = config['teams']['news']

In [39]:
def send_facebook(savename,link,title,summary):
  datetime.today()
  today = datetime.today().strftime('%m월%d일')
  
  token = make_array(tips,';')
  graph = facebook.GraphAPI(token[1])
  
  message = '{}\n#앤비팁스 #Test Google Colab #(출처)플래텀 #{} #실시간뉴스 #{} #{}'.format(link,today,title,summary)
  graph.put_photo(image=open(IMG_URI+savename, 'rb'),message=message)

In [40]:
def send_teams(link,title,summary):
  datetime.today()
  today = datetime.today().strftime('%m월%d일')
  
  message = '{}\n#앤비팁스 #Test Google Colab #(출처)플래텀 #{} #실시간뉴스 #{} #{}'.format(link,today,title,summary)
  token = make_array(news,';')
  
  headers = {'Content-Type': 'Application/JSON;charset=utf-8',}
  card = 'MessageCard'
  color = '0072C6'
  title = '최신 IT 뉴스'
  
  data = '{"@context":"'+token[0]+'","@type":"'+card+'","themeColor":"'+color+'","title":"'+title+'","text":"'+message+'"}'
  response = requests.post(token[1], headers=headers, data=data.encode('utf-8'))	  	

In [41]:
def compare_seq(csv_seq,seq):
  if csv_seq == seq:
    return 1
  else:
    return 0

In [42]:
def make_array(string,keyword): 
  arr = []
  arr = string.split(keyword)
  return arr	

In [43]:
def make_date(string):
  numbers = re.findall('\d+', string)
  _year = numbers[0]
  _month = numbers[1]
  _day = numbers[2]

  return _year+'.'+_month+'.'+_day

In [44]:
def remove_keyword(string):
  string = string.replace('\'', '')
  string = string.replace('\"', '')
  string = string.replace('[', '')
  string = string.replace(']', '') 
  string = string.replace('\r', '') 
  string = string.replace('\n', '')      
  return string  

In [45]:
def download_image(seq,src):
  savename = seq+'.png'
  urllib.request.urlretrieve(src, IMG_URI+savename)
  
  with Image.open(IMG_URI+savename) as image:
    image.save(IMG_URI+savename, quality=40)

  return savename

In [46]:
def write_csv(df):
  savename = CSV_URI+'platum.csv'
  
  tmp = []
  tmp = savename.split('/')
  tmp2 = tmp[len(tmp)-1]
  
  if os.path.exists(savename):
    print('Add data', tmp2)
    df_read = pd.read_csv(savename, header=None)
    
    last_row = df_read.tail(1)
    csv_seq = last_row.iloc[:,0]
    result = compare_seq(int(csv_seq.values[0]),int(df['seq'].values[0]))
  else :
    print('Make file', tmp2)
    result = 0
    
  if result:
    print('Overlap contents!!!')
  else:
    df.to_csv(savename, header=False, index=False, mode='a', encoding='utf-8-sig')
  
  return result

In [47]:
def scrappy(soup):

  seq, title, summary, src, link, rdate = [], [], [], [], [], []

  contents = soup.select('div.sidebar_content div.post_filter_wrapper div div.post.type-post div.post_wrapper div.post_content_wrapper div.post_header.search')
  # print(contents)

  try:
    tmp = contents[0].select('div.post_img.static.one_third a')[0].get('href')
    tmp = tmp.split("/")[-1]
    seq.append(tmp)
  except ValueError:
    seq.append('seq')    

  try:
    tmp = contents[0].select('div.post_img.static.one_third a')[0].get('href')
    link.append(tmp)
  except ValueError:
    link.append('link')

  try:
    tmp = contents[0].select('div.post_img.static.one_third a img')[0].get('src')
    src.append(tmp)
  except ValueError:
    src.append('src')    

  try:
    tmp = contents[0].select('div.post_header_title.two_third.last h5 a')[0].text
    title.append(tmp)
  except ValueError:
    title.append('title') 

  try:
    tmp = contents[0].select('div.post_header_title.two_third.last p')[0].text.strip()
    summary.append(remove_keyword(tmp))
    # print(summary)
  except ValueError:
    summary.append('summary') 

  try:
    tmp = contents[0].select('div.post_header_title.two_third.last div.post_detail.post_date span.post_info_date')[0].text.strip()
    rdate.append(make_date(tmp))
    # print(rdate)
  except ValueError:
    rdate.append('rdate')     


  dic_news= {}

  dic_news['seq'] = seq
  dic_news['link'] = link
  dic_news['src'] = src
  dic_news['title'] = title
  dic_news['summary'] = summary
  dic_news['rdate'] = rdate

  # print(dic_news)

  df_news = pd.DataFrame(dic_news)
  # print(df_news)

  result = write_csv(df_news)
  
  if result:
    print('Overlap contents !!!')
  else:
    savename = download_image(seq[0],src[0])
    send_facebook(savename,link[0],title[0],summary[0])
    send_teams(link[0],title[0],summary[0])  

In [48]:
url = 'https://platum.kr'

resp = requests.get(url)

soup = BeautifulSoup(resp.text, 'lxml')

scrappy(soup)

Make file platum.csv
